In [1]:
import csv
import time
from collections import defaultdict, Counter
import json
from urllib.request import Request, urlopen
import pickle

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import shape, Point
import shapefile
from geopy import distance
import plotly.figure_factory as ff
import plotly.io as pio
import plotly as plt
pio.renderers.default = "jupyterlab"
import plotly.express as px
import plotly.graph_objects as go

In [3]:
from tqdm import tqdm

In [4]:
from utils import *

In [ ]:
df = load_csv_export_dataset("export_dataframe_stage2.csv")

In [6]:
df['wbd'].value_counts()

Proctor Creek-Chattahoochee River    3682
Peachtree Creek                      3611
Utoy Creek                           2724
March Creek-Chattahoochee River      1865
South Fork Peachtree Creek           1772
                                     ... 
Douglas Lake-Big Slough                 1
Clyatt Mill Creek                       1
Mill Creek-Alapaha River                1
Troublesome Creek-Towaliga River        1
Simpson Creek                           1
Name: wbd, Length: 564, dtype: int64

In [7]:
locs = df[['SiteName', 'SiteLocation']]

In [8]:
df = df[~df['SiteLocation'].isna()]

In [9]:
#locs = locs[locs['SiteLocation'] != 'NULL']
locs = locs[~locs['SiteLocation'].isna()]

In [10]:
loc_pairs, geo_locs, gdf = get_loc_objects_from_series(locs['SiteLocation'])

In [11]:
loc_lookup = fetch_geo_locs()

In [12]:
# watershed boundaries
WBD_gj = shapefile.Reader("hydrologic_units\wbdhu12_a_ga.shp").__geo_interface__

In [13]:
GA_map = get_state_map()

In [14]:
wbd_names = set([wbd['properties']['name'] for wbd in WBD_gj['features']])

---

In [15]:
df_map = pd.DataFrame(data={'name': [wbd['properties']['name'] for wbd in WBD_gj['features']]})

In [24]:
# df_map.head()

In [25]:
df['Counts'] = df.groupby(['wbd'])['wbd'].transform('count')
df.head(2)

,group_rid,GroupName,site_rid,state,county,lat,long,SiteName,SiteLocation,event_rid,...,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings,wbd,Counts
0,840,Rails to Trails,18,Georgia,Hall,34.4103,-83.8462,West Fork of Little River,34.4103 -83.8462,92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Fork Little River,6.0
1,840,Rails to Trails,18,Georgia,Hall,34.4103,-83.8462,West Fork of Little River,34.4103 -83.8462,93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Fork Little River,6.0


In [18]:
df2 = df

In [19]:
df_graph = df.iloc[:, [-2,-1]]
df_graph = df_graph.rename(columns={"wbd": "name"})
df_graph = df_graph.drop_duplicates()
df_graph.head()

,name,Counts
0,West Fork Little River,6.0
2,Turkey Creek-Appalachee River,19.0
16,Wrights Creek-Alcovy River,7.0
19,Baldwin Creek-Bear Creek,2.0
20,Bear Creek,165.0


In [23]:
wbd_graph = df_map.set_index('name').join(df_graph.set_index('name'), how='outer', on='name')

In [21]:
wbd_graph['name'] = wbd_graph.index
wbd_graph.head()

,name,Counts
Cane Water Pond,Cane Water Pond,2.0
Hutchinson Mill Creek,Hutchinson Mill Creek,15.0
Caney Branch-Satilla River,Caney Branch-Satilla River,110.0
Town of Mount Vernon-Oconee River,Town of Mount Vernon-Oconee River,NaN
Dasher Creek-Savannah River,Dasher Creek-Savannah River,3.0


---

In [ ]:
fig = px.choropleth_mapbox(wbd_graph, #array/object with wbd names in the same order as the json file
                           geojson=WBD_gj,
                           locations="name",
                           color="Counts",
                           featureidkey="properties.name",
                           center={"lat": 32.8407, "lon": -83.6324}, # macon
                           mapbox_style="stamen-terrain", #"carto-positron",
                           color_continuous_scale="Jet", #Sunsetdark, Jet, 
                           zoom=6,
                           opacity=0.7,
                      )


fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig.show()